# Регрессия

С 1ого октября отменяется НДС на бриллианты, теперь они становятся новым инвестиционным инструментом. Давайте сделаем модель ценообразования для них.

Скачайте датасет diamonds.csv

В нем представлены характеристики бриллиантов и их цены с сайта jamesallen (B2C площадка) с 2022-07-01

**Описание полей**


* fluor - флуорисценуия (свойство камня светиться в лучах ультр)
* symmetry - показатель симметричности
* platform - название платформы, где был размещен камень
* quality_group - составной показатель из cut polish symmetry
* size_group - каратно весовая группа
* big_size_group - каратно-весовая группа
* shape - форма
* color - цвет
* clarity - чистота
* cut - качество огранки (может быть только у круглых камней)
* polish - полировка
* id - номер камня
* date - дата
* price - цена
* carat - кол-во карат
* tablepercent - размер площадки по отношению ширине
* price_per_carat - цена за карат
* z - длина (diameter)
* x - ширина
* depth_perc - отношение высоты к ширине
* y - высота



In [1]:
import pandas as pd
import numpy as np

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# path = 'drive/MyDrive/'

Mounted at /content/drive


In [3]:
# df = pd.read_csv(path + 'diamonds.csv', index_col=0)

In [2]:
df = pd.read_csv('diamonds.csv', index_col=0)

Хоти предсказать price_per_carat

## Задание 1: Очистка

Не все камни успевают продаться за один месяц, поэтому в таблице есть повторы. Объедините данные по одному камню: подумайте, как лучше это сделать, какую цену брать.

Попробуйте найти аномалии: вдруг цена на некоторые камни сильно меняется (то есть продавец сам не знает, по какой цене их продавать). Также убедитесь, что остальные параметры камня не меняются.

In [232]:
id_stat = df.groupby(by='id')['carat'].count().to_frame() #Посчитайте статистику по кол-ву камней

# разделите выборку на две части
id_count_1 = id_stat[id_stat['carat'] == 1]
df_one = df.query('id in @id_count_1.index')  # те камни, которые встречались один раз
id_count_2_3 = id_stat[(id_stat['carat'] == 2) + (id_stat['carat'] == 3)]
df_three = df.query('id in @id_count_2_3.index') # те камни, которые встречались 2 или 3 раза


stat = df_three.groupby('id')['price', 'carat', 'price_per_carat', 'z', 'x', 'depth_perc', 'y'].agg(np.std) # Посчитайте дстандартное отклонение по по нескольким полям,
#  подумайте где оно должно равняться 0, а где меняться в каких-то разумных пределах
anomaly_ids = list(stat[(stat['carat'] > 0) + (stat['z'] > 0) + (stat['x'] > 0) + (stat['depth_perc'] > 0) + (stat['y'] > 0)].index)
iqr = stat['price_per_carat'].quantile(0.75) - stat['price_per_carat'].quantile(0.25)
stat[stat['price_per_carat'] > 1.5 * stat['price_per_carat'].quantile(0.75)].index
anomaly_ids.append(stat[stat['price_per_carat'] > 1.5 * stat['price_per_carat'].quantile(0.75)].index)

# Удалите аномальные наблюдения
df_three = df_three.query('id not in @anomaly_ids').sort_values(by=['id','date'])

df_three = df_three.groupby('id').agg('last').reset_index() #тепреь в качестве цены возьмем последнее значение по времени

df = pd.concat([df_one, df_three]).reset_index(drop=True) # соединяем результаты

C:\Users\ogloc\AppData\Local\Temp\ipykernel_22060\403455913.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stat = df_three.groupby('id')['price', 'carat', 'price_per_carat', 'z', 'x', 'depth_perc', 'y'].agg(np.std) # Посчитайте дстандартное отклонение по по нескольким полям,


Цены на бриллианты достаточно сильно меняются, попробуйте вычислить коэффициент инфляции и привести цены к последнему месяцу. Стоит учесть, что цены на все бриллианты не изменяются синхронно, то есть изменение в определенных группах может быть разным.

Определите эти группы и рассчитайте коэффициенты инфляции для каждой из них. Подправьте цены на эти коэффициенты и создайте новую переменную.

In [ ]:
categories = ['...'] #определите по каким группам отпределять инфляцию (к примеру можно добавить каратную группу)

df_index = df.groupby(['date'] + categories)[['price_per_carat']]\
                                                                .mean()\
                                                                    .reset_index() # индекс цен

date_max = df_index.date.max()


df_index = df_index.merge(df_index.query('date == @date_max')[categories + ['price_per_carat']]\
                          .rename(columns={'price_per_carat': 'price_per_carat_max'}), 
                          on=categories, how='outer') # сопоставляем группы с максимальной датой

df_index['inflation'] = df_index.['...'] / df_index.price_per_carat # вычисляем инфляцию

df_with_inf = df.merge(df_index[['date'] + categories + ['inflation']], 
                        on=['date'] + categories, how='left') # соединяем все в одной таблице

## Задание 2: Модель

Определите функцию потерь (MSE или MAE) и аргументируйте выбор.
Попробуйте сделать baseline.
Используйте LableEncoder для категориальных фичей и постройте линейную модель.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LableEncoder
from sklearn.metrics import ...


X = df.drop(columns=["..."])
y = df["..."]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

Теперь попробуйте OHE или TargetEncoder (сравните их).

Нормализуйте данные.

Поработайте с пропусками (обратите внимание на то, что у fluor возможен пропуск значения, а возможно отсутствие флуоресценции)

Покажите, насколько получилось улучшить результат.

In [124]:
!pip install category_encoders

     ---------------------------------------- 72.4/72.4 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 233.8/233.8 kB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 9.1/9.1 MB 10.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

from typing import Tuple, List


def OHE(df: pd.DataFrame, columns: List[str]) -> Tuple[pd.DataFrame, List[str]]:
    index = df.index
    one = OneHotEncoder(sparse=False, categories='auto')
    ohe = one.fit_transform(df[columns])
    col_names = one.get_feature_names(input_features = columns)
    df = df.drop(columns, axis=1)
    df = df.reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame(ohe, columns=col_names)], axis = 1)
    df = df.set_index(index)
    return (df, col_names)

scaler = StandardScaler() # Помните, что на тесте делаем только transform
  

Сравните с KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor



# Классификация

Загрузите датасет bodyPerformance

Описание:

https://www.kaggle.com/datasets/kukuroo3/body-performance-data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = 'drive/MyDrive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv(path + '/Classroom/DS 11 VI/ДЗ-регрессия_классификация/bodyPerformance.csv') #укажите свой путь

## Задание 1: Определение гендера

Постройте модель, которая будет определять гендер.

Для этого определите две метрики качества, на которые вы будете смотреть. Аргументируйте свой выбор.

После чего преобразуйте категориальные переменные и постройте модель бинарной классификации на основе линейного классификатора и сравните с KNN.

P.S.: не забудте про нормализацию.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

X = df.drop(columns=["..."])
y = df["..."]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

scaler = StandardScaler()

## Задание 2: Определение класса

Теперь опробуем построить модель, которая будет предсказывать class физической формы.

Для этого определите метрики качества для задачи мультиклассификации (аргументируйте выбор).

Постройте модель мультиклассовой классификации на основе линейного классификатора и сравните с KNN.

In [ ]:
X = df.drop(columns=["..."]) # укажите новый таргет
y = df["..."]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)